### Blocks World

An extension of blocks world with two agents.
- one can only moves blocks marked with vowels, the other with consonants.

The goal is to arrange the blocks according to the goal.

In [6]:
from src.llm_plan.LLM import GPT_Ollama
from src.llm_plan.Parser import PDDLParser
from src.llm_plan.Problem import ProblemStaticBlocksworld
from src.llm_plan.StaticEnvironment import StaticBlocksworld

model = GPT_Ollama()
env = StaticBlocksworld(num_vowels=3, num_consonants=3, easy=True)
problem = ProblemStaticBlocksworld(env)
parser = PDDLParser()

env.render()

G    I    Q    S    A    E


In [ ]:
# Generate the PDDL plans for each agent
agents = ["Agent A", "Agent B"]
agent_prompts = {agent: "" for agent in agents}
agent_responses = {agent: "" for agent in agents}

for agent in agents:
    print(f"Prompting {model.model_name} with the following prompt:\n")
    print(problem.prompts[agent])
    agent_prompts[agent] = f"{agent} Problem:\n" + problem.prompts[agent] + "\n"
    answer = model.generate_sync(problem.system_prompts[agent], 
                                 problem.prompts[agent])
    print("Answer:")
    print(answer)
    agent_responses[agent] = f"{agent} Response:\n" + answer + "\n"
    print("-"*30 + "\n")

In [ ]:
# Prompt the orchestrator for the final plan
orchestrator_prompt = problem.prompts["Orchestrator"].format(pddl_agent_A=agent_prompts["Agent A"] + agent_responses["Agent A"],
                                                             pddl_agent_B=agent_prompts["Agent B"] + agent_responses["Agent B"],
                                                             goal=env.goal["Agent A"])

final_plan = model.generate_sync(problem.system_prompts["Orchestrator"], 
                                 orchestrator_prompt)

print(final_plan)                       

In [ ]:
# Save the final plan to a file
with open("./tmp/final_plan_in_text.pddl", "w") as f:
    f.write(final_plan)

# Isolate the pddl problem and domain
pddl_domain, pddl_problem = parser.parse("./test/data/sample_pddl.txt")

In [ ]:
# Save the PDDL domain and problem to files
with open("./tmp/domain.pddl", "w") as domain_file:
    domain_file.write(pddl_domain)

with open("./tmp/problem.pddl", "w") as problem_file:
    problem_file.write(pddl_problem)


In [ ]:
import subprocess

# Invoke fast downward and solve the problem
command = "./solvers/fast-downward-24.06.1/fast-downward.py --alias lama-first --plan-file ./tmp/sas_plan ./tmp/domain.pddl ./tmp/problem.pddl > ./tmp/logs.txt 2>&1"

subprocess.run(command, shell=True)